# Visualizations

## Config

In [ ]:
%config InlineBackend.figure_format = 'svg'

## Imports

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

from matplotlib.axes import Axes
from matplotlib.figure import Figure

import seaborn as sns
import seaborn.objects as so

## Aesthetics

In [ ]:
sns.set_theme(
    style='whitegrid'
)

## Load data

In [ ]:
os.chdir('/Users/mattia/Documents/Projects/RAG-Fact-Checking/src')
from src.reports import *

In [ ]:
folder_path_prefix = '/Users/mattia/Desktop/Lab avanzato 1 - RAG/Results'

In [ ]:
folder_name = '20241011-212456-UTC'
file_name = 'metrics' + '.csv'
input_file_path = os.path.join(folder_path_prefix, folder_name, file_name)
metrics_df = pd.read_csv(input_file_path, index_col=0)

In [ ]:
folder_name = '20241011-212456-UTC'
file_name = 'raw_data' + '.csv'
input_file_path = os.path.join(folder_path_prefix, folder_name, file_name)
raw_data_df = pd.read_csv(input_file_path, index_col=0)

## FIX DATA

In [ ]:
def fix_target(row: pd.Series) -> int:
    if row['DATASET_NAME'] == 'climate_fever':
        return 1 - row['target']
    else:
        return row['target']

raw_data_df['target'] = raw_data_df.apply(fix_target, axis=1)

In [ ]:
rb = ReportBuilderFor2ClassificationLevels()

In [ ]:
# Simulate real run to rebuild the reports with the correct values

# Initialize vars
reports = []
raw_data = []
n_fixed = 0

for llm_name in ['mistral-nemo:12b-instruct-2407-fp16', 'llama3.1:8b-instruct-fp16']:
    # Update config params
    config.LLM_NAME = llm_name
    for dataset_name in ['climate_fever']:
        # Update config params
        config.DATASET_NAME = dataset_name
        if dataset_name == 'climate_fever':  # Missing '.pqt' file for this dataset, so we use '.csv' instead
            config.GROUND_TRUTH_DATASET_PATH = os.path.join(config.DATASET_PATH_PREFIX, config.DATASET_NAME, 'ground_truth.csv')
        else:
            config.GROUND_TRUTH_DATASET_PATH = os.path.join(config.DATASET_PATH_PREFIX, config.DATASET_NAME, 'ground_truth.pqt')
        config.ALL_EVIDENCE_VECTOR_STORE_PATH = os.path.join(config.DATASET_PATH_PREFIX, config.DATASET_NAME, 'embeddings/512/')
        for levels in [2, 6]:
            if (levels == 6) and (dataset_name in ['climate_fever', 'feverous']):
                continue  # These datasets do not support the 6 classification levels
            # Update config params
            config.CLASSIFICATION_LEVELS = levels
            for fill, invert in [(True, False), (False, True), (False, False)]:
                # Update config params
                config.FILL_EVIDENCE = fill
                config.FILL_EVIDENCE_UPPER_LIMIT = 10
                config.INVERT_EVIDENCE = invert
                for i in range(1, 11):
                    # Update config params
                    config.TRUNCATED_RANKING_RETRIEVER_RESULTS = i

                    # Subset raw data
                    raw_data_df_subset = raw_data_df[(raw_data_df.LLM_NAME == llm_name) & (raw_data_df.DATASET_NAME == dataset_name) & (raw_data_df.CLASSIFICATION_LEVELS == levels) & (raw_data_df.FILL_EVIDENCE == fill) & (raw_data_df.INVERT_EVIDENCE == invert) & (raw_data_df.TRUNCATED_RANKING_RETRIEVER_RESULTS == i)]
                    
                    # Subset metrics (only for assertion check)
                    metrics_df_row = metrics_df[(metrics_df.LLM_NAME == llm_name) & (metrics_df.DATASET_NAME == dataset_name) & (metrics_df.CLASSIFICATION_LEVELS == levels) & (metrics_df.FILL_EVIDENCE == fill) & (metrics_df.INVERT_EVIDENCE == invert) & (metrics_df.TRUNCATED_RANKING_RETRIEVER_RESULTS == i)]
                    
                    # Compute correct values
                    rb._predictions = raw_data_df_subset['prediction']
                    rb._targets = raw_data_df_subset['target']
                    rb_df_row = rb.build()
                    
                    # Replace values
                    for val_name in ['accuracy', 'precision', 'recall', 'f1']:
                        metrics_df.loc[(metrics_df.LLM_NAME == llm_name) & (metrics_df.DATASET_NAME == dataset_name) & (metrics_df.CLASSIFICATION_LEVELS == levels) & (metrics_df.FILL_EVIDENCE == fill) & (metrics_df.INVERT_EVIDENCE == invert) & (metrics_df.TRUNCATED_RANKING_RETRIEVER_RESULTS == i), val_name] = rb_df_row[val_name].iloc[0]
                        
                    # Assertion check
                    assert len(raw_data_df_subset) == (metrics_df_row['n_total'].iloc[0] - metrics_df_row['n_undefined_prediction'].iloc[0])
                    
                    n_fixed += 1
                    

In [ ]:
# Additional check of correctness
assert len(metrics_df[metrics_df.DATASET_NAME == 'climate_fever']) == n_fixed

## Write fixes to files

In [ ]:
folder_name = '20241011-212456-UTC-fixed_climate_fever'

In [ ]:
output_file_path = os.path.join(folder_path_prefix, folder_name, 'raw_data.csv')
raw_data_df.to_csv(output_file_path)

In [ ]:
output_file_path = os.path.join(folder_path_prefix, folder_name, 'metrics.csv')
metrics_df.to_csv(output_file_path)